# Large Circuit Partitioning Benchmark
This notebook builds composite circuits to exercise QuASAr's partitioning capabilities.


In [ ]:
import os
os.environ['QUASAR_QUICK_MAX_QUBITS'] = '16'
os.environ['QUASAR_QUICK_MAX_GATES'] = '200'
os.environ['QUASAR_QUICK_MAX_DEPTH'] = '20'


In [ ]:
from benchmarks.extensive_circuits import dual_ghz_qft_circuit, adder_ghz_qaoa_circuit
from quasar.analyzer import CircuitAnalyzer
from quasar.simulation_engine import SimulationEngine

ghz_qft = dual_ghz_qft_circuit(width=64)
hybrid = adder_ghz_qaoa_circuit(bit_width=32, qaoa_layers=3)


In [ ]:
import time

engine = SimulationEngine()

def _format_subsystems(subsystems):
    if not subsystems:
        return '-'
    return ', '.join('(' + ', '.join(str(q) for q in group) + ')' for group in subsystems)

def describe_partitions(name, partitions):
    if not partitions:
        print(f"[{name}] No partitions identified.")
        return
    for idx, part in enumerate(partitions, start=1):
        backend_name = getattr(part.backend, 'name', str(part.backend))
        history_len = len(part.history)
        subsystem_desc = _format_subsystems(part.subsystems)
        print(
            f"[{name}] Partition {idx}: backend={backend_name}, subsystems=[{subsystem_desc}], gates={history_len}"
        )

def describe_conversions(name, conversions):
    if not conversions:
        print(f"[{name}] No conversion layers.")
        return
    for layer in conversions:
        src = getattr(layer.source, 'name', str(layer.source))
        tgt = getattr(layer.target, 'name', str(layer.target))
        print(
            f"[{name}] Conversion {layer.primitive}: boundary={layer.boundary}, {src}→{tgt}, rank={layer.rank}, frontier={layer.frontier}"
        )

def prepare_circuit(circuit, name):
    print(f"[{name}] Starting analysis...")
    start = time.perf_counter()
    analyzer = CircuitAnalyzer(circuit, estimator=engine.planner.estimator)
    analysis = analyzer.analyze()
    print(f"[{name}] Analysis finished in {time.perf_counter() - start:.2f}s.")
    print(f"[{name}] Preparing execution plan...")
    start = time.perf_counter()
    plan = engine.scheduler.prepare_run(circuit, analysis=analysis)
    duration = time.perf_counter() - start
    total_steps = len(getattr(plan, 'explicit_steps', plan.steps))
    print(f"[{name}] Plan prepared in {duration:.2f}s with {total_steps} step(s).")
    describe_partitions(name, list(circuit.ssd.partitions))
    describe_conversions(name, list(circuit.ssd.conversions))
    return analysis, plan

ghz_qft_analysis, ghz_qft_plan = prepare_circuit(ghz_qft, 'GHZ-QFT')
hybrid_analysis, hybrid_plan = prepare_circuit(hybrid, 'Adder-GHZ-QAOA')


In [ ]:
from itertools import count

def simulate_prepared(circuit, plan, analysis, name):
    total_steps = len(getattr(plan, 'explicit_steps', plan.steps))
    step_counter = count(1)

    def monitor(step, observed, estimated):
        idx = next(step_counter)
        backend_name = getattr(step.backend, 'name', str(step.backend))
        gate_count = step.end - step.start
        est_time = getattr(estimated, 'time', 0.0)
        obs_time = getattr(observed, 'time', 0.0)
        print(
            f"[{name}] Step {idx}/{total_steps} on {backend_name}: {gate_count} gate(s), est. {est_time:.2f}s, observed {obs_time:.2f}s."
        )

    print(f"[{name}] Executing prepared plan with {total_steps} step(s)...")
    conversions = list(getattr(plan, 'explicit_conversions', ()))
    if conversions:
        print(f"[{name}] Planned conversions:")
        describe_conversions(name, conversions)
    ssd, metrics = engine.scheduler.run(
        circuit,
        plan,
        analysis=analysis,
        instrument=True,
        monitor=monitor,
    )
    runtime = metrics.cost.time
    peak_memory = (metrics.cost.memory / (1024 ** 3)) if metrics.cost.memory else 0.0
    print(
        f"[{name}] Simulation complete in {runtime:.2f}s with peak memory {peak_memory:.2f} GiB and {metrics.backend_switches} backend switch(es)."
    )
    print(f"[{name}] Final SSD partitions: {len(ssd.partitions)}")
    return ssd, metrics

ghz_qft_ssd, ghz_qft_metrics = simulate_prepared(ghz_qft, ghz_qft_plan, ghz_qft_analysis, 'GHZ-QFT')
hybrid_ssd, hybrid_metrics = simulate_prepared(hybrid, hybrid_plan, hybrid_analysis, 'Adder-GHZ-QAOA')
